In [ ]:
# Import relevant packages
import os

import xarray as xr
import pandas as pd
import numpy as np

import calliope
from calliope.core import time

# internal package which includes functions for scenario reduction
import utils

os.chdir(os.path.join('..', 'model'))

## Mean model

In [ ]:
# Mean model can be built directly, with clustering applied within Calliope
mean_model = calliope.Model('model.yaml', override_file='overrides.yaml:dmuu')

## Individual scenario

In [ ]:
## Load a scenario, e.g. scenario 1, into the model

## Note: this is long-winded as we have stored our CSVs as Gzip files on GitHub (for storage capacity reasons).
## If you have uncompressed the CSVs locally then remove the _prep_csv and CSV purging steps

def get_scenario_model(scenario, probability=None):
    
    def _prep_csv(filename):  # unzip GZip & save as CSV
        pd.read_csv(
            os.path.join('timeseries_data', 'DMUU', 'scenarios', filename), 
            compression='gzip', header=0, index_col=0, parse_dates=True
        ).to_csv(os.path.join('timeseries_data', 'DMUU', 'scenarios', filename.replace('.gz', '.csv')))
        print('Prepped CSV:', filename.replace('.gz', ''))
        
    if isinstance(scenario, list): # we can provide a list of scenarios, for a multi-scenario Calliope model
        scenario = {i: 1/len(scenario) for i in scenario}
    if isinstance(scenario, dict): # a scenario dict gives `scenario_number:probability` as `key:value` pair
        for energy in ['heat', 'electricity']:
            for i in scenario.keys():
                _prep_csv('demand_{}_{}.gz'.format(energy, i))
        calliope.AttrDict({
            int(k): {'techs': {'demand_heat': {'constraints': {'resource': 'file=scenarios/demand_heat_{}.csv'.format(k)}},
                          'demand_electricity': {'constraints': {'resource': 'file=scenarios/demand_electricity_{}.csv'.format(k)}}},
                'model': {'probability': v}}
            for k, v in scenario.items()
        }).to_yaml(os.path.join('timeseries_data', 'DMUU', 'scenarios.yaml'))
        
        scenario_model = calliope.Model(
            'model.yaml', override_file='overrides.yaml:dmuu', 
            scenario_file=os.path.join('timeseries_data', 'DMUU', 'scenarios.yaml:'+','.join([str(i) for i in scenario]))
        )
        
    else:
        overrides = {}
        for energy in ['heat', 'electricity']:
            _prep_csv('demand_{}_{}.gz'.format(energy, scenario))

            key = 'techs.demand_{}.constraints.resource'.format(energy)
            overrides[key] = 'file=scenarios/demand_{}_{}.csv'.format(energy, scenario)

        scenario_model = calliope.Model('model.yaml', override_file='overrides.yaml:dmuu', override_dict=overrides)
        
    # Purge CSVs following preparing the scenario model:
    print('Purging CSVs')
    for file in os.listdir(os.path.join('timeseries_data', 'DMUU', 'scenarios')):
        if file.endswith(".csv"):
            os.remove(os.path.join('timeseries_data', 'DMUU', 'scenarios', file))
            
    return scenario_model

scenario_model = get_scenario_model(scenario=1)

## Scenario reduction

In [ ]:
# To undertake scenario reduction, run all scenarios independently, then load them into one xarray Dataset:
all_scenarios = xr.concat(
    [xr.open_dataset('path_to_scenario_model_{}'.format(i) for i in range(500))], 
    dim=pd.Index(data=[i for i in range(500)], name='scenarios'),
    data_vars='different'
)

# Then run scenario reduction
reduced_scenarios = utils.get_reduced_scenarios(all_scenarios.cost.values, 16)
reduced_scenarios_df = utils.get_redistributed_probabilities(all_scenarios.cost.values, reduced_scenarios)

## Scenario optimisation

In [ ]:
# Create multi-scenario model, for scenario optimisation
scenarios = pd.read_csv(os.path.join('timeseries_data', 'DMUU', 'reduced_scenarios.csv'), header=0, index_col=0)
scenario_model = get_scenario_model({int(i): scenarios.loc[i].probability for i in scenarios.reduced_scenario.unique()})

# Add SO-related attributes
scenario_model._model_data.attrs['run.mode'] = 'scenario_plan'
scenario_model._model_data.attrs['run.alpha'] = '1'
scenario_model._model_data.attrs['run.beta'] = '0'

# Edit solver, if necessary
#scenario_model._model_data.attrs['run.solver'] = ''

# Recommended to save this to file, for running on a remote cluster:
#scenario_model.to_netcdf('scenario_model_DMUU.nc')

# If you want to run right here:
#scenario_model.run()